# Getting HRA ASCT+B Table data


In [81]:
# Install pre-requisite libraries
!pip install requests

## Imports / functions


In [82]:
import requests
import json


def fetch_json(purl):
    response = requests.get(purl, headers={"Accept": "application/json"})
    return json.loads(response.text)


def is_asctb_table(purl):
    return (
        purl.startswith("https://purl.humanatlas.io/asct-b/")
        and "crosswalk" not in purl
    )


def get_latest_asctb_data():
    hra_collection = fetch_json("https://purl.humanatlas.io/collection/hra")
    digital_objects = hra_collection["metadata"]["had_member"]
    tables = {}
    for purl in sorted(filter(is_asctb_table, digital_objects)):
        table_name = purl.split("/")[-2]
        table_data = fetch_json(purl)
        table_rows = table_data["data"]["asctb_record"]
        tables[table_name] = table_rows
    return tables

## Get the latest HRA ASCT+B table data


In [83]:
tables = get_latest_asctb_data()
print("Fetched", len(tables), "ASCT+B tables")

Fetched 33 ASCT+B tables


## Inspect the Kidney table


In [84]:
kidney = tables["kidney"]
print("The kidney table has", len(kidney), "rows")

The kidney table has 78 rows


In [85]:

def format_term(str):
    return str.replace("https://purl.org/ccf/ASCTB-TEMP_", "ASCTB-TEMP:")


def simple_list(elts):
    return [f'{format_term(x["source_concept"])} ({x["ccf_pref_label"]})' for x in elts]


print("1st Row of Kidney Table")
record = kidney[0]

print("\nAnatomical Structures:")
print(simple_list(record["anatomical_structure_list"]))

print("\nCell Types:")
print(simple_list(record["cell_type_list"]))

print("\nGene Biomarkers:")
print(simple_list(record.get("gene_marker_list", [])))

print("\nReferences:")
print(record["references"])

1st Row of Kidney Table

Anatomical Structures:
['UBERON:0002113 (kidney)', 'UBERON:0002015 (kidney capsule)']

Cell Types:
['CL:0000499 (stromal cell)', 'ASCTB-TEMP:capsule-mesenchymal-stromal-cell (capsule mesenchymal stromal cell)']

Gene Biomarkers:
['HGNC:3802 (Foxd1)', 'HGNC:1975 (VSX2)']

References:
['https://doi.org/10.1371/journal.pone.0187118']
